In [ ]:
import SSLimPy.interface.sslimpy as sslimpy
import numpy as np
import astropy.units as u
from copy import copy
import matplotlib.pyplot as plt
import seaborn as sns
import astropy
from getdist.gaussian_mixtures import GaussianND
from getdist import plots

color_palette = sns.color_palette("colorblind")
color_palette

# Compute Cosmology at the fiducial value

In [ ]:
cosmodict={"h":0.6774,
           "Omegam":0.31,
           "Omegab":0.05,
           "sigma8":0.82,
           "mnu":0.06,
           "ns":0.967}

fidBAO_dict = {
    "alpha_par": 1,
    "alpha_perp": 1,
    "sigmav": 3 * u.Mpc,
    "bmean": 1.7,
    "Tmean": 1.5 * u.uK,
    }

settings = {"code":"class",
            "do_RSD" : True,
            "nonlinearRSD" : True,
            "FoG_damp" : "ISTF_like",
            "halo_model_PS" : True,
            "Smooth_window" : True,
            "output" : ["Power spectrum"],
            "TracerPowerSpectrum":"clustering",
            "kmin": 1.e-3 * u.Mpc**-1,
            "kmax": 0.1 * u.Mpc**-1,
            "nk": 100,
            }

In [ ]:
SSLIMPY = sslimpy.sslimpy(settings_dict=settings,
                           cosmopars=cosmodict,
                           BAOpars=fidBAO_dict, 
                           specifications="input/survey_files/forecast_test.yaml")

Just to show how to do a fisher forecast we choose following free parameters

$\left\lbrace\alpha_\parallel, \alpha_\perp,  \sigma_v, T_{\rm CO}, b_{\rm CO}, \sigma_8, \right\rbrace$

Obtain fiducial power spectrum and covariance

In [ ]:
Fiducial_output = SSLIMPY.compute(cosmopars=cosmodict,
                                  astropars=dict(),
                                  BAOpars=fidBAO_dict,
                                  output=["Power spectrum",
                                          "Covariance"])


Check the fiducial power spectra and (inverse) covariance

In [ ]:
Ps_fid = Fiducial_output["Power spectrum"]
PfidMoments = np.array([Ps_fid.Pk_0bs, Ps_fid.Pk_2bs, Ps_fid.Pk_4bs]) * Ps_fid.Pk_0bs.unit
k = Ps_fid.k

In [ ]:
colors= iter(color_palette)
for i in range(3):
    c = next(colors)
    plt.loglog(k,PfidMoments[i,:,0], c=c)
    plt.loglog(k,-PfidMoments[i,:,0], ls="--", c=c)


In [ ]:
derivative_dic = dict()
eps = 1e-2

# Cosmo Derivs

In [ ]:
cosmo_free_pars = ["sigma8"]

for param in cosmo_free_pars:
    cosmo_var_dict = copy(cosmodict)
    
    # do step
    cosmo_var_dict[param] = cosmodict[param] * ( 1 + eps)
    # compute power spectrum
    Pobs = SSLIMPY.compute(cosmopars=cosmo_var_dict,
                                    astropars=dict(),
                                    BAOpars=fidBAO_dict,
                                    )["Power spectrum"]
    dPp = np.array([Pobs.Pk_0bs, Pobs.Pk_2bs, Pobs.Pk_4bs]) * Pobs.Pk_Obs.unit

    cosmo_var_dict[param] = cosmodict[param] * ( 1 - eps)
    Pobs = SSLIMPY.compute(cosmopars=cosmo_var_dict,
                                    astropars=dict(),
                                    BAOpars=fidBAO_dict,
                                    )["Power spectrum"]
    dPm = np.array([Pobs.Pk_0bs, Pobs.Pk_2bs, Pobs.Pk_4bs]) * Pobs.Pk_Obs.unit

    derivative_dic[param] = (dPp - dPm) / (2 * eps * cosmodict[param])

# Nuisance Derivs

In [ ]:
nuiscance_free_pars = [ "alpha_par", "alpha_perp", "sigmav", "bmean", "Tmean"]

for param in nuiscance_free_pars:
    BAO_var_dict = copy(fidBAO_dict)
    
    # do step
    BAO_var_dict[param] = BAO_var_dict[param] * (1 + eps)
    # compute power spectrum
    Pobs = SSLIMPY.compute(cosmopars=cosmodict,
                           astropars=dict(),
                           BAOpars=BAO_var_dict,
                           )["Power spectrum"]
    dPp = np.array([Pobs.Pk_0bs, Pobs.Pk_2bs, Pobs.Pk_4bs]) * Pobs.Pk_Obs.unit

    BAO_var_dict[param] = BAO_var_dict[param] * (1 - eps)
    Pobs = SSLIMPY.compute(cosmopars=cosmodict,
                           astropars=dict(),
                           BAOpars=BAO_var_dict,
                           )["Power spectrum"]
    dPm = np.array([Pobs.Pk_0bs, Pobs.Pk_2bs, Pobs.Pk_4bs]) * Pobs.Pk_Obs.unit

    derivative_dic[param] = (dPp - dPm) / (2 * eps * fidBAO_dict[param])

# Construct Fisher matrix 

In [ ]:
allpars = np.array([*cosmo_free_pars, *nuiscance_free_pars])

In [ ]:
Cov = Fiducial_output["Covariance"].gaussian_cov()
invCov = np.empty(Cov.shape, dtype=u.Quantity)
z = Pobs.z
for iz in range(len(z)):
    invCov[:,:,:,iz] = np.linalg.inv(Cov[:,:,:,iz])
invCov = invCov /Cov.unit

In [ ]:
Fisher = np.empty((*allpars.shape,*allpars.shape), dtype=u.Quantity)
for i, p1 in enumerate(allpars):
    for j, p2 in enumerate(allpars):
        # The index for k is k, ell is i, ell' is j
        # Covaraiance has indixes k, i, j
        # The derives have indixes i/j, k
        # At the end every index is summed over
        Fij = np.sum(
            np.einsum(
                "ik...,kij...,jk...",
                derivative_dic[p1], 
                invCov,
                derivative_dic[p2],
            )
        )
        Fisher[i,j] = Fij

In [ ]:
_unitless = np.empty((*allpars.shape,*allpars.shape))
for i, p1 in enumerate(allpars):
    for j, p2 in enumerate(allpars):
        _unitless[i,j] = Fisher[i,j].value

In [ ]:
invFisher = np.linalg.inv(_unitless)

In [ ]:
invFisher

Plot the Fisher lets see

In [ ]:
mean = [0.82, 1, 1, 3, 1.7, 1.5]


In [ ]:
print(mean)
print(allpars)
print([np.sqrt(invFisher[i,i]) for i in range(_unitless.shape[0])])

In [ ]:
colors = [sns.color_palette('colorblind')[ii] for ii in [2]]

In [ ]:
gauss1=GaussianND(mean=mean, cov=_unitless ,names=allpars ,is_inv_cov=True)
g = plots.get_subplot_plotter(rc_sizes=True,subplot_size = 6,subplot_size_ratio= 1,width_inch=6)
g.triangle_plot([gauss1],filled=[False],params=allpars,legend_labels=[r'Default'],contour_lws=[1],colors=colors, contour_colors=colors)